In [2]:
import hvplot.pandas
import numpy as np
import param
import pandas as pd
import panel as pn
import panel.widgets as pnw
import sys

import util_panel
dir_analysis = '/home/jovyan/local-climate-data-tool/Analysis/Phase1_ProcessData'
sys.path.insert(0, dir_analysis)
import analysis_parameters

# Define Global Variables

In [3]:
### DEFINE GLOBAL VARIABLES ###
# Change this directory to be wherever you put the dummy data
DATA_PATH = "/home/smturbev/uwsed/proj/"
# Keys for reading in data files (.zarr)
THIS_EXPERIMENT_ID = ['historical','ssp126', 'ssp370','ssp245','ssp585']
# Keys for dictionary with observational data too
EXPERIMENT_KEYS = THIS_EXPERIMENT_ID.copy()
EXPERIMENT_KEYS.append('historical_obs')

DICT_LATLON = util_panel.create_country2city2latlon_dict() # Courtesy of https://simplemaps.com/data/world-cities
COLORS = ['black','blue','green','orange','red','gray']
DICT_COLORS = dict(zip(EXPERIMENT_KEYS, COLORS))

# Read in the data with a function

In [3]:
DICT_TS = util_panel.read_data()

# Generate Panel

In [4]:
def hvplot(df, lat, lon, **kwargs):
    """Generates plot -- parameter of time_series function below"""
    colors=['black','blue','green','orange','red','gray']
    hv0 = df[0].hvplot.line(alpha=1.0, color=colors[0], label=EXPERIMENT_KEYS[0]) 
    hv1 = df[1].hvplot.line(alpha=0.5, color=colors[1], label=EXPERIMENT_KEYS[1])
    hv2 = df[2].hvplot.line(alpha=0.5, color=colors[2], label=EXPERIMENT_KEYS[2]) 
    hv3 = df[3].hvplot.line(alpha=0.5, color=colors[3], label=EXPERIMENT_KEYS[3]) 
    hv4 = df[4].hvplot.line(alpha=0.5, color=colors[4], label=EXPERIMENT_KEYS[4]) 
    hv5 = df[5].hvplot.line(alpha=1.0, color=colors[5], label=EXPERIMENT_KEYS[5])
    ymax = df[4].max()+3
    ymin = df[0].min()-3
    return (hv0 * hv1 * hv2 * hv3 * hv4 * hv5).opts(width=1000, height=350, legend_position='top', 
                                                    ylabel='Temperature (K)', ylim = (ymin,ymax),
                                                    title='Location: %dN, %dE'%(lat, lon))

def time_series(lat, lon, annual_mean, view_fn=hvplot):
    """Generate interactive portions of longitude and latitude for data"""
    ds = util_panel.read_data()
    df = [None]*6
    for i,exp in enumerate(EXPERIMENT_KEYS):
        one_ds = ds[exp]
        data_to_plot = one_ds.sel(lat=lat, lon=lon, method='nearest')
        if annual_mean:
            df[i] = data_to_plot.groupby('time.year').mean().to_dataframe()['mean']
        else:
            df[i] = data_to_plot.to_dataframe()['mean']
    return view_fn(df, lat, lon)

In [5]:
def hvplot1(df, df_hist, scenario, lat, lon, **kwargs):
    """Generates plot -- parameter of time_series function below"""
    colors = ['black','blue','green','orange','red','gray']
    hv1 = df['mean'].hvplot.line(color='black', legend=False)
    hv2 = df[['min','max']].hvplot.area(y='min', y2='max', color=DICT_COLORS[scenario], alpha=0.5, legend=False)
    hv3 = df_hist['mean'].hvplot.line(color='black', legend=False)
    hv4 = df_hist[['min','max']].hvplot.area(y='min', y2='max', alpha=0.5, color=DICT_COLORS['historical'], legend=False)
    ymax = df['max'].max() + 3
    ymin = df_hist['min'].min() - 3
    return (hv1 * hv2 * hv3 * hv4).opts(width=1000, height=300, ylabel='Temperature (K)',
                                        title="Scenario (%s) and Historical model range for %dN, %dE"%(scenario,lat,lon))
def time_series1(scenario, lat, lon, annual_mean, view_fn=hvplot1):
    """Generate interactive portions of longitude and latitude for data"""
    ds = util_panel.read_data()
    one_ds = ds[scenario]
    data_to_plot = one_ds.sel(lat=lat, lon=lon, method='nearest')
    dfh = ds['historical'].sel(lat=lat, lon=lon, method='nearest')
    if annual_mean:
        df = data_to_plot.groupby('time.year').mean().to_dataframe()
        df_hist = dfh.groupby('time.year').mean().to_dataframe() 
    else:
        df = data_to_plot.to_dataframe()
        df_hist = dfh.groupby.to_dataframe() 
    return view_fn(df, df_hist, scenario, lat, lon)

In [6]:
class Scenario(param.Parameterized):
    """Builds a time series of model min, max and mean for a specified scenario"""
    def __get__(self, instance, owner):
        return instance.scenario
    def __set__(self, instance, value):
        instance.scenario = value

In [7]:
class Selector(param.Parameterized):
    """Builds a timeseries plot of scenarios based on user input"""
    country = param.ObjectSelector(default='United States', objects=sorted(list(DICT_LATLON.keys())))
    city = param.ObjectSelector(default='Seattle', objects=sorted(list(DICT_LATLON['United States'].keys())))
    custom = param.Boolean(False, doc='customize latitude and longitude')
    latitude = param.Number(0, bounds=(-90, 90))
    longitude = param.Number(180, bounds=(0, 360))
    annual_mean = param.Boolean(True, doc='show annual mean, averages out annual variation')
    scenario = param.ObjectSelector(default='ssp585', objects=['ssp126', 'ssp370', 'ssp245', 'ssp585'])
    scenario_obj = Scenario()
    
    @param.depends('country', watch=True)
    def _update_cities(self):
        cities = sorted(list(DICT_LATLON[self.country].keys()))
        self.param['city'].objects = cities
        self.city = cities[0]
          
    @param.depends('scenario','city','latitude','longitude','custom','annual_mean')
    def view1(self):
        if self.custom:
            lat, lon = self.latitude, self.longitude
        else:
            lat, lon = DICT_LATLON[self.country][self.city]
        return time_series1(self.scenario, lat, lon, self.annual_mean)
    
    @param.depends('city','longitude','latitude','annual_mean','custom')
    def view(self): 
        if self.custom:
            lat, lon = self.latitude, self.longitude
        else:
            lat, lon = DICT_LATLON[self.country][self.city]
        return time_series(lat, lon, self.annual_mean)
    
timeseries_obj = Selector()

In [8]:
plots_obj = pn.Column(timeseries_obj.view, timeseries_obj.view1)
panel_obj = pn.Row(timeseries_obj.param, plots_obj)
web = pn.Column('# Local Climate Tool', panel_obj)

In [8]:
web.show()

In [ ]:
web.serveable()